In [35]:
import requests
import os
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

True

In [36]:
os.environ.get('EIA_API_KEY')

'JtlyggzhwQyEktgzvkWJ4rhwYzA3nUJHfBfiHu0w'

In [18]:
url = 'https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/'
params = {
    'frequency': 'hourly',
    'data[0]': 'value',
    'start': '2024-05-06T00',
    'end': '2024-05-07T00',
    'sort[0][column]': 'period',
    'sort[0][direction]': 'desc',
    'offset':'0',
    'api_key': os.environ.get('EIA_API_KEY')

}

In [20]:
response = requests.get('https://api.eia.gov/v2/electricity/rto/fuel-type-data/data', params=params)

In [45]:
df = pd.DataFrame(response.json()['response']['data'])
df = df[df['respondent']=='MISO'].reset_index(drop=True)

In [46]:
df

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2024-05-07T00,MISO,"Midcontinent Independent System Operator, Inc.",SUN,Solar,2168,megawatthours
1,2024-05-07T00,MISO,"Midcontinent Independent System Operator, Inc.",WND,Wind,15928,megawatthours
2,2024-05-07T00,MISO,"Midcontinent Independent System Operator, Inc.",WAT,Hydro,2696,megawatthours
3,2024-05-07T00,MISO,"Midcontinent Independent System Operator, Inc.",NG,Natural gas,32674,megawatthours
4,2024-05-07T00,MISO,"Midcontinent Independent System Operator, Inc.",COL,Coal,16296,megawatthours
...,...,...,...,...,...,...,...
86,2024-05-06T12,MISO,"Midcontinent Independent System Operator, Inc.",NG,Natural gas,25605,megawatthours
87,2024-05-06T12,MISO,"Midcontinent Independent System Operator, Inc.",OTH,Other,267,megawatthours
88,2024-05-06T12,MISO,"Midcontinent Independent System Operator, Inc.",WAT,Hydro,1359,megawatthours
89,2024-05-06T12,MISO,"Midcontinent Independent System Operator, Inc.",NUC,Nuclear,8689,megawatthours


In [53]:
fuel_sources = df['type-name'].unique()
df2 = pd.concat([df.set_index(['period']).groupby('type-name')['value'].get_group(key) for key in fuel_sources],axis=1)
df2.columns = fuel_sources
df2 = df2.reset_index()

In [54]:
fuel_sources

array(['Solar', 'Wind', 'Hydro', 'Natural gas', 'Coal', 'Nuclear',
       'Other'], dtype=object)

In [55]:
df2

,period,Solar,Wind,Hydro,Natural gas,Coal,Nuclear,Other
0,2024-05-07T00,2168,15928,2696,32674,16296,8620,294
1,2024-05-06T23,4138,15406,2791,31995,15543,8619,278
2,2024-05-06T22,4768,13985,2531,32209,15466,8621,282
3,2024-05-06T21,4939,13948,2265,31870,15219,8626,280
4,2024-05-06T20,4925,14386,1634,31553,14977,8634,284
5,2024-05-06T19,4964,14884,1724,30990,14770,8638,292
6,2024-05-06T18,4813,14455,1847,30210,15036,8644,286
7,2024-05-06T17,4588,14776,1651,29319,14813,8656,272
8,2024-05-06T16,4207,15085,1430,28352,14459,8668,277
9,2024-05-06T15,3476,15380,1687,27393,14383,8666,277
